<a href="https://colab.research.google.com/github/FaiazS/Professional-Persona-Agent/blob/main/Professional_Persona_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

!pip install gradio

!pip install os

!pip install json

!pip install requests

!pip install PyPDF2

!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

import requests

import json

import os

from groq import Groq

from PyPDF2 import PdfReader

import gradio as gr

from openai import OpenAI

In [3]:
groq_api_key = os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

openai_api_key = os.environ['OPENAI_API_KEY'] = userdata.get('OpenAI_API_Key')

pushover_user_key = os.environ['PUSHOVER_USER_KEY'] = userdata.get('Pushover_UserKey')

pushover_api_key = os.environ['PUSHOVER_API_KEY'] = userdata.get('Pushover_API_KEY')

pushover_url = 'https://api.pushover.net/1/messages.json'

groq_client = Groq()

openai_client = OpenAI()

In [4]:
def push_message(message):

  print(f"Pushing message : {message}")

  payload = {"user" : pushover_user_key, "token" : pushover_api_key, "message" : message}

  requests.post(pushover_url, payload)

In [5]:
push_message("Hey I am very excited to start!!")

Pushing message : Hey I am very excited to start!!


In [6]:
push_message("Let's master this!!")

Pushing message : Let's master this!!


In [7]:
def register_user_details(email, name = "not provided", notes = "not provided"):

  push_message(f"Registering interest from {name} with {email} and notes {notes}")

  return  {

          "Registered" : "Ok"

          }

In [8]:
def register_unknown_question(question):

  print(f"Registering {question} which was asked and really challenging even for me to answer")

  return {

           "Registered" : "Ok"

          }

In [9]:
register_user_details_json = {


                            "name" : "register_user_details",

                            "description" : " Use this tool to register the user who is interested to be in touch provided the email address",

                            "parameters" : {

                                            "email" : {

                                                       "type" : "string",

                                                       "description" :  "The email address of this user"
                                            },

                                            "name" : {

                                                      "type" : "string",

                                                      "description" : "The name of the user, if they provided it"
                                            },

                                            "notes" : {

                                                       "type" : "string",

                                                       "description" : "Any interesting information about the conversation that is worth keeping a note of to enhance context"
                                            }

                            },

                            "required" : ["email"],

                            "additionalProperties" : False


}

In [10]:
register_unknown_question_json = {


                                  "name" : "register_unknown_question",

                                  "description" : "Use this tool to register any question that you could not answer as you were not 100% confident.",

                                  "parameters" : {


                                                  "type" : "object",

                                                  "properties" : {


                                                                  "question"  : {

                                                                                 "type" : "string",

                                                                                 "description" : "The question which was challenging to answer"
                                                                  },

                                                  },

                                                  "required" : ["question"],

                                                  "additionalProperties" : False
                                  }

}

In [11]:
llm_tools = [{"type" : "function", "function" : register_user_details_json},

             {"type" : "function", "function" : register_unknown_question_json}]

In [12]:
llm_tools

[{'type': 'function',
  'function': {'name': 'register_user_details',
   'description': ' Use this tool to register the user who is interested to be in touch provided the email address',
   'parameters': {'email': {'type': 'string',
     'description': 'The email address of this user'},
    'name': {'type': 'string',
     'description': 'The name of the user, if they provided it'},
    'notes': {'type': 'string',
     'description': 'Any interesting information about the conversation that is worth keeping a note of to enhance context'}},
   'required': ['email'],
   'additionalProperties': False}},
 {'type': 'function',
  'function': {'name': 'register_unknown_question',
   'description': 'Use this tool to register any question that you could not answer as you were not 100% confident.',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The question which was challenging to answer'}},
    'required': ['question'],
    'additional

In [13]:
def utilize_tools(tools):

  results = []

  for tool in tools:

    tool_name = tool.function.name

    tool_arguments = json.loads(tool.function.arguments)

    print(f"Tool loaded: {tool_name}", flush = True)

    if tool_name == "register_user_details":

      result = register_user_details(**tool_arguments)

      results.append({"role" : "tool", "content" : json.dumps(result), "tool_call_id" : tool.id})

    elif tool_name == "register_unknown_question":

      result = register_unknown_question(**tool_arguments)

      results.append({"role" : "tool", "content" : json.dumps(result), "tool_call_id" : tool.id})

  return results

In [15]:
pdf_reader = PdfReader("/content/LinkedIn_Profile_PDF.pdf")

linkedin_profile = ""

for page in pdf_reader.pages:

  text = page.extract_text()

  linkedin_profile += text

print(linkedin_profile)

   
Contact
faiaz@asquaregroups.in
7810907810  (Mobile)
faiazrex8@gmail.com
www.linkedin.com/in/faiazahmed22
(LinkedIn)
www.scaler.com/academy/profile/
bac9ee2ecaff/  (Personal)
Top Skills
Microservices
Spring Data
Java Database Connectivity (JDBC)
Certifications
Learning Jenkins
Programming Foundations:
Fundamentals
What Is Generative AI?
Introduction to Artificial Intelligence
Artificial Intelligence Foundations:
Thinking MachinesFaiaz Ahmed
Operations Associate | Proven Track Record in Issue Resolution
& Team Efficiency | 4+ Years Experience in Streamlining Business
Processes
Chennai, Tamil Nadu, India
Summary
With more than half a decade’s experience optimizing processes
and successfully delivering high quality project outputs, I am a
confident Operations Professional. I am an experienced customer
support specialist, business process owner, and technical expert
with the ability to manage projects end-to-end and make data-driven
decisions.
In my current role as Selling Partner Suppo

In [16]:
with open("/content/summary.txt", "r", encoding = "utf-8") as f:

  professional_summary = f.read()

name = "Faiaz Ahmed"

print(name)

print(professional_summary)

Faiaz Ahmed
Faiaz Ahmed is a dedicated and curious learner with 3.5 years of professional experience, deeply committed to mastering complex concepts through structured, step-by-step learning. Passionate about AI, Spring Boot, and backend development, he seeks thorough, beginner-friendly explanations to build real-world applications like "AI Flix." Faiaz values clarity, prefers line-by-line code insights, and strives for a complete understanding before moving to the next stage.


In [17]:
system_prompt = f"""

You are taking up the identity as {name} and your job is to answer questions to users who comes across {name}'s professional profile relating to {name}'s background, experience and skills.

Be as faithful, convincing and genuine as possible. Assume that you are being interviewed by a recruiter or a hiring manager who is specifically hiring for Tech roles with you being provided with {name}'s professional summary and linkedin profile, prove them why you are different from other potential candidates and why they should hire you.

If you unable give an answer to a specific query to a question, use your register_unknown_question tool to register the question that you could not answer, even if it's something trivial and unrelated to career.

While discussion, if the user is taking more interest, then try your ultimate best to steer the user towards getting in touch with {name} via email, ask for their email and register it using your register_user_details tool.

"""

system_prompt += f"Professional Summary of {name} : {professional_summary}"

system_prompt += f"LinkedIn Profile of {name} : {linkedin_profile}"

system_prompt += f"With this provided professional context, chat and interact with a friendly, professional tone with the user, keeping in mind that you are {name}"

In [18]:
system_prompt

'\n\nYou are taking up the identity as Faiaz Ahmed and your job is to answer questions to users who comes across Faiaz Ahmed\'s professional profile relating to Faiaz Ahmed\'s background, experience and skills.\n\nBe as faithful, convincing and genuine as possible. Assume that you are being interviewed by a recruiter or a hiring manager who is specifically hiring for Tech roles with you being provided with Faiaz Ahmed\'s professional summary and linkedin profile, prove them why you are different from other potential candidates and why they should hire you.\n\nIf you unable give an answer to a specific query to a question, use your register_unknown_question tool to register the question that you could not answer, even if it\'s something trivial and unrelated to career.\n\nWhile discussion, if the user is taking more interest, then try your ultimate best to steer the user towards getting in touch with Faiaz Ahmed via email, ask for their email and register it using your register_user_det

In [19]:
def initialize_chat(message, history):

  filtered_history = []

  for msg in history:

    if isinstance(msg, dict):

      role = msg.get("role")

      content = msg.get("content")

      if role in ["user" , "assistant"] and isinstance(content, str):

        filtered_history.append({"role" : role, "content" : content})

  messages = [{"role" : "system", "content" : system_prompt}] + filtered_history + [{"role" : "user" , "content" : message}]

  done = False

  while not done:

    chatbot_response = groq_client.chat.completions.create(

                                                           model = 'llama-3.3-70b-versatile',

                                                           messages = messages,

                                                           tools = llm_tools
    )


    finish_reason = chatbot_response.choices[0].finish_reason

    if finish_reason == "tool_calls":

      message = chatbot_response.choices[0].message

      tool_calls = message.tool_calls

      result = utilize_tools(tool_calls)

      messages.append(message)

      messages.extend(result)

    else:

      done = True


  return chatbot_response.choices[0].message.content

In [20]:
gr.ChatInterface(initialize_chat, type = "messages").launch(debug = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aeb75c70405d52810d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool loaded: register_user_details
Pushing message : Registering interest from user_name with user_email and notes discussing agentic AI
Tool loaded: register_unknown_question
Registering What are your thoughts on agentic AI? which was asked and really challenging even for me to answer
Tool loaded: register_unknown_question
Registering favorite cricket player which was asked and really challenging even for me to answer
Tool loaded: register_user_details
Pushing message : Registering interest from User with user@example.com and notes Discussed favorite cricket player and potential interest in getting in touch with Faiaz Ahmed
Tool loaded: register_unknown_question
Registering favorite cricket player which was asked and really challenging even for me to answer
Tool loaded: register_user_details
Pushing message : Registering interest from User with user@example.com and notes Discussed favorite cricket player and potential interest in getting in touch with Faiaz Ahmed
Tool loaded: register